In [99]:
import json
import numpy as np
import math
from collections import deque
from confluent_kafka import Consumer, Producer
import tensorflow as tf


In [100]:
# 1. 모델 로드
MODEL_PATH = "model.keras"
model = tf.keras.models.load_model(MODEL_PATH)

# 2. 스케일러(mean, std) 로드
SCALER_PATH = "scaler_mean_std.npz"
scaler_data = np.load(SCALER_PATH)
mean = scaler_data["mean"]
std = scaler_data["std"]

In [91]:
# =============================
# 1. Kafka 설정
# =============================
kafka_conf = {
    "bootstrap.servers": "kafka.dongango.com:9094",
    "group.id": "drowny-lstm-consumer",
    "auto.offset.reset": "earliest"
}

input_topic = "sess-wphp1s2v-metnfuq9"
output_topic = input_topic + "-LSTM"

# level: -1,0,1,2,3  -> 총 5클래스. (-1 포함)
CLS_LEVELS = [-1, 0, 1, 2, 3]

In [92]:
FEATURE_COLS = ["EAR", "MAR", "yawn_rate_per_min", "blink_rate_per_min",
                "avg_blink_dur_sec", "longest_eye_closure_sec"]

SEQ_LEN = 150   # 5초 @ 30fps
HOP = 30        # 1초 단위 슬라이딩

In [93]:
def fit_standardizer(X: np.ndarray):
    """
    채널별 표준화 스케일러(평균/표준편차) 반환.
    """
    # X shape: (N, T, C)
    mean = X.reshape(-1, X.shape[-1]).mean(axis=0)
    std = X.reshape(-1, X.shape[-1]).std(axis=0)
    std = np.where(std < 1e-8, 1.0, std)
    return mean.astype(np.float32), std.astype(np.float32)

def apply_standardizer(X: np.ndarray, mean: np.ndarray, std: np.ndarray):
    return (X - mean) / std

In [94]:
# =============================
# 4. 윈도우 버퍼 (deque 사용)
# =============================
window = deque(maxlen=SEQ_LEN)
frame_numbers = deque(maxlen=SEQ_LEN)

def preprocess_value(value_dict):
    """필요한 feature만 뽑아 float 배열로 변환"""
    return [float(value_dict.get(col, np.nan)) for col in FEATURE_COLS]

def predict_and_publish():
    """버퍼에서 예측 후 Kafka publish"""
    if len(window) == SEQ_LEN:
        X = np.array(window).reshape(1, SEQ_LEN, len(FEATURE_COLS))
        
        # ✅ 표준화 적용
        X = apply_standardizer(X, mean, std)

        y_pred = model.predict(X, verbose=0)
        class_index = int(np.argmax(y_pred, axis=1)[0])   # multi-class softmax 결과
        drowny_level = CLS_LEVELS[class_index]
        
        result = {
            "frame": int(frame_numbers[-1]),  # 마지막 frame 번호
            "drowny-level": drowny_level
        }
        
        producer.produce(
            output_topic,
            key="drowny-lstm-result",
            value=json.dumps(result)
        )
        producer.flush()
        print("Published:", result)

In [95]:
# =============================
# 3. Kafka Consumer / Producer 초기화
# =============================
consumer = Consumer(kafka_conf)
consumer.subscribe([input_topic])

producer = Producer({"bootstrap.servers": kafka_conf["bootstrap.servers"]})

In [97]:
def transform_json(input_json: dict) -> dict:
    # timestamp_ms -> time_sec 변경
    output = {
        "frame": input_json.get("frame"),
        "time_sec": input_json.get("timestamp_ms"),
    }

    # metrics 값 병합
    metrics = input_json.get("metrics", {})
    for k, v in metrics.items():
        if k == "mar":  # "mar" 키 제외
            continue
        # NaN 값 0으로 치환
        if isinstance(v, float) and math.isnan(v):
            output[k] = 0
        else:
            output[k] = v

    return output

In [89]:
# 사용 예시
input_data = {
    "topic" : "sess-9kt6byvo-mev3xjib",
    "frame" : 2,
    "source_idx" : 2,
    "timestamp_ms" : 1756367650176,
    "rel_time_sec" : 0.033333,
    "metrics" : {
        "label_name" : "eyes_state/open",
        "EAR" : 0.3197801223283421,
        "MAR" : 0.006035359592953227,
        "yawn_rate_per_min" : 0,
        "blink_rate_per_min" : 0,
        "avg_blink_dur_sec" : float("NaN"),
        "longest_eye_closure_sec" : float("NaN"),
        "ear_left" : 0.3188832691201519,
        "ear_right" : 0.3206769755365323,
        "ear_mean" : 0.3197801223283421,
        "mar" : 0.006035359592953227,
        "face_found" : True
    }
}

result = transform_json(input_data)
print(result)

{'frame': 2, 'time_sec': 1756367650176, 'label_name': 'eyes_state/open', 'EAR': 0.3197801223283421, 'MAR': 0.006035359592953227, 'yawn_rate_per_min': 0, 'blink_rate_per_min': 0, 'avg_blink_dur_sec': 0, 'longest_eye_closure_sec': 0, 'ear_left': 0.3188832691201519, 'ear_right': 0.3206769755365323, 'ear_mean': 0.3197801223283421, 'face_found': True}


In [98]:
# =============================
# 5. 스트리밍 loop
# =============================
try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        if msg.error():
            print("Kafka error:", msg.error())
            continue

        value = transform_json(json.loads(msg.value().decode("utf-8")))
        frame_no = int(value.get("frame"))

        # 순서 체크 (frame 번호가 이전보다 작으면 skip)
        if frame_numbers and frame_no <= frame_numbers[-1]:
            continue

        # feature 추출 및 버퍼에 저장
        features = preprocess_value(value)
        window.append(features)
        frame_numbers.append(frame_no)

        # 윈도우가 다 차면 예측
        if len(window) == SEQ_LEN:
            predict_and_publish()

            # 1초(30 frame)씩 슬라이딩 → 앞부분 drop
            for _ in range(HOP):
                if window:
                    window.popleft()
                    frame_numbers.popleft()

except KeyboardInterrupt:
    print("Stopped by user")
finally:
    consumer.close()

Published: {'frame': 150, 'drowny-level': 0}
Published: {'frame': 180, 'drowny-level': 3}
Published: {'frame': 210, 'drowny-level': 3}
Published: {'frame': 240, 'drowny-level': 3}
Published: {'frame': 270, 'drowny-level': 3}
Published: {'frame': 300, 'drowny-level': 3}
Published: {'frame': 330, 'drowny-level': 3}
Published: {'frame': 360, 'drowny-level': 3}
Published: {'frame': 390, 'drowny-level': 3}
Published: {'frame': 420, 'drowny-level': 3}
Published: {'frame': 450, 'drowny-level': 3}
Published: {'frame': 480, 'drowny-level': 3}
Published: {'frame': 510, 'drowny-level': 3}
Published: {'frame': 540, 'drowny-level': 3}
Published: {'frame': 570, 'drowny-level': 3}
Published: {'frame': 600, 'drowny-level': 3}
Published: {'frame': 630, 'drowny-level': 3}
Published: {'frame': 660, 'drowny-level': 3}
Published: {'frame': 690, 'drowny-level': 3}
Published: {'frame': 720, 'drowny-level': 3}
Published: {'frame': 750, 'drowny-level': 3}
Published: {'frame': 780, 'drowny-level': 3}
Published: